In [1]:
import requests
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# For normal

In [2]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url="https://countries.trevorblades.com/")

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

# Provide a GraphQL query


# Execute the query on the transport
# result = client.execute(query)
# print(result)


# Async for ipython

In [3]:
import asyncio

from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

transport = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-matic")

## Graph Queries

### Transactions from User

In [4]:
query = gql(
    """
query GetSenderTransactions($userId: String!, $sendingChainId: BigInt!, $status: TransactionStatus) {
  transactions(
    where: {user: $userId, status: $status, sendingChainId: $sendingChainId}
    orderBy: preparedBlockNumber
    orderDirection: desc
  ) {
    id
    status
    chainId
    preparedTimestamp
    user {
      id
    }
    router {
      id
    }
    receivingChainTxManagerAddress
    sendingAssetId
    receivingAssetId
    sendingChainFallback
    receivingAddress
    callTo
    sendingChainId
    receivingChainId
    callDataHash
    transactionId
    amount
    expiry
    preparedBlockNumber
    bidSignature
    prepareCaller
    fulfillCaller
    cancelCaller
    prepareTransactionHash
    fulfillTransactionHash
    cancelTransactionHash
  }
}
"""
)

params = {
        "sendingChainId": 137,
        "userId": "0x4752a32843b661fedc525c979aaa01fa54d31366",
        "status": "Fulfilled"
    }

In [5]:
query = gql(
    """
query fetchAllTransactions($batch: Int!) {
  transactions (
    first:1000, skip: $batch,
    orderBy: preparedBlockNumber
    orderDirection: asc
  ){
    id
    status
    chainId
    preparedTimestamp
    user {
      id
    }
    router {
      id
    }
    receivingChainTxManagerAddress
    sendingAssetId
    receivingAssetId
    sendingChainFallback
    receivingAddress
    callTo
    sendingChainId
    receivingChainId
    callDataHash
    transactionId
    amount
    expiry
    preparedBlockNumber
    bidSignature
    prepareCaller
    fulfillCaller
    cancelCaller
    prepareTransactionHash
    fulfillTransactionHash
    cancelTransactionHash
  }
}
"""
)

params = {
        "batch": 0
    }

### Fetching Routers data

In [185]:
query = gql(
    """
query fetchAllTransactions($batch: Int!) {
  routers(first:100, skip: $batch) {
    id
    assetBalances {
      id
      amount
    }
  }
}
"""
)

params = {
        "batch": 0
    }

## Querying and Fetching data

In [186]:
transport_matic = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-matic")
transport_bsc = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-bsc")
transport_xdai = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-xdai")
transport_fantom = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-fantom")

In [179]:
txn_columns = ['amount', 'bidSignature', 'callDataHash', 'callTo', 'cancelCaller', 'cancelTransactionHash', 'chainId', 'expiry', 'fulfillCaller', 'fulfillTransactionHash', 'id', 'prepareCaller', 'prepareTransactionHash', 'preparedBlockNumber', 'preparedTimestamp', 'receivingAddress', 'receivingAssetId', 'receivingChainId', 'receivingChainTxManagerAddress', 'router', 'sendingAssetId', 'sendingChainFallback', 'sendingChainId', 'status', 'transactionId', 'user']

matic_txns = pd.DataFrame(columns=txn_columns)
bsc_txns = pd.DataFrame(columns=txn_columns)
xdai_txns = pd.DataFrame(columns=txn_columns)
fantom_txns = pd.DataFrame(columns=txn_columns)

In [187]:
router_columns = [ "router_id", "asset_id", "amount"]

matic_routers = pd.DataFrame(columns=router_columns)
bsc_routers = pd.DataFrame(columns=router_columns)
xdai_routers = pd.DataFrame(columns=router_columns)
fantom_routers = pd.DataFrame(columns=router_columns)

In [209]:
async def fetch_chain_transactions(query, params, transport, dataframe):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport, fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        for batch in range(42): #Just a random no. It has to be just > 6. 
            result = await session.execute(query, variable_values=params)
            for tr in result["transactions"]:
                list_values = list(tr.values())
                list_values[19] = list_values[19]["id"]
                list_values[25] = list_values[25]["id"]
                dataframe.loc[len(dataframe.index)] = list_values
            if len(result["transactions"]) < 1000:
                break
            params['batch']+=1000
            print(params["batch"], end="-")
        print("Done")
        params["batch"] = 0
        return result

In [189]:
async def fetch_routers(query, params, transport, dataframe):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport, fetch_schema_from_transport=True,
    ) as session:
        per_batch = 100
        # Execute single query
        for batch in range(42): #Just a random no. It has to be just > 6. 
            result = await session.execute(query, variable_values=params)
            for router in result["routers"]:
                for asset in router["assetBalances"]:
                    list_values = []
                    list_values.append(router["id"])
                    list_values.append(asset["id"].split("-")[0])
                    list_values.append(asset["amount"])
                    dataframe.loc[len(dataframe.index)] = list_values
            if len(result["routers"]) < per_batch:
                break
            params['batch']+=per_batch
            print(params["batch"], end="-")
        print("Done")
        params["batch"] = 0

In [8]:
await fetch_chain_transactions(query, params, transport_matic, matic_txns)
await fetch_chain_transactions(query, params, transport_bsc, bsc_txns)
await fetch_chain_transactions(query, params, transport_xdai, xdai_txns)
await fetch_chain_transactions(query, params, transport_fantom, fantom_txns)

1000-2000-3000-4000-
1000-2000-3000-

1000-


In [188]:
await fetch_routers(query, params, transport_matic, matic_routers)
await fetch_routers(query, params, transport_bsc, bsc_routers)
await fetch_routers(query, params, transport_xdai, xdai_routers)
await fetch_routers(query, params, transport_fantom, fantom_routers)

Done
Done
Done
Done


{'routers': [{'assetBalances': [{'amount': '13359446320',
     'id': '0x04068da6c83afcfa0e13ba15a6696662335d5b75-0x29a519e21d6a97cdb82270b69c98bac6426cdcf9'},
    {'amount': '5495546996',
     'id': '0x049d68029688eabf473097a2fc38ef61633a3c7a-0x29a519e21d6a97cdb82270b69c98bac6426cdcf9'},
    {'amount': '19620460444725527916447',
     'id': '0x8d11ec38a3eb5e956b052f67da8bdc9bef8abf3e-0x29a519e21d6a97cdb82270b69c98bac6426cdcf9'}],
   'id': '0x29a519e21d6a97cdb82270b69c98bac6426cdcf9'},
  {'assetBalances': [{'amount': '3986796605000000000',
     'id': '0x8d11ec38a3eb5e956b052f67da8bdc9bef8abf3e-0x709a10a487aa2fc85845b9c8a56ad28d5fa44914'}],
   'id': '0x709a10a487aa2fc85845b9c8a56ad28d5fa44914'},
  {'assetBalances': [{'amount': '4916432055000000000',
     'id': '0x8d11ec38a3eb5e956b052f67da8bdc9bef8abf3e-0xaca3ba0cf90431fccb0f03e045d362350a9455b1'}],
   'id': '0xaca3ba0cf90431fccb0f03e045d362350a9455b1'},
  {'assetBalances': [{'amount': '0',
     'id': '0x04068da6c83afcfa0e13ba15a669666233

## Basic Chain Stats

In [10]:
matic_txns[matic_txns.sendingChainId=="137"]["status"].value_counts()

Fulfilled    2982
Cancelled     174
Prepared       20
Name: status, dtype: int64

In [9]:
bsc_txns[bsc_txns.sendingChainId=="56"]["status"].value_counts()

Fulfilled    1106
Cancelled     183
Prepared       47
Name: status, dtype: int64

In [11]:
xdai_txns[xdai_txns.sendingChainId=="100"].shape

(325, 26)

In [12]:
fantom_txns[fantom_txns.sendingChainId=="250"].shape

(334, 26)

# Analysing data

## Txns Data

### Combining all data

In [44]:
import json
with open('chain_asset_data.json', 'w') as fp:
    json.dump(chain_asset_data, fp)

In [14]:
matic_txns["chain"] = "Polygon"

bsc_txns["chain"] = "BSC"

xdai_txns["chain"] = "xDai"

fantom_txns["chain"] = "Fantom"

In [15]:
two_sided_txns = pd.concat([matic_txns,bsc_txns,xdai_txns,fantom_txns])

In [16]:
two_sided_txns.shape

(10260, 27)

In [17]:
two_sided_txns["txn_type"] = two_sided_txns.apply(lambda x: "single" if x["sendingChainId"] == x["chainId"] else "repeat", axis=1)

In [18]:
one_sided_txns = two_sided_txns[two_sided_txns["txn_type"]=="single"].copy(deep=True)

In [86]:
one_sided_txns.reset_index(drop=True,inplace=True)

### Asset Movement

In [20]:
chain_mapping = {"56": "BSC", "137": "Polygon", "250": "Fantom", "100": "xDai"}

In [21]:
def transacting_chains(row):
    val = chain_mapping[row["sendingChainId"]] + " -> " + chain_mapping[row["receivingChainId"]]
    return val

In [231]:
two_sided_txns['asset_movement'] = two_sided_txns.apply(transacting_chains, axis=1)
one_sided_txns['asset_movement'] = one_sided_txns.apply(transacting_chains, axis=1)
one_sided_txns['asset_movement'].value_counts()

Polygon -> BSC       2162
BSC -> Polygon       1076
Polygon -> Fantom     763
Polygon -> xDai       251
BSC -> Fantom         213
xDai -> Polygon       210
Fantom -> Polygon     191
Fantom -> BSC         114
xDai -> Fantom         60
xDai -> BSC            55
BSC -> xDai            47
Fantom -> xDai         29
Name: asset_movement, dtype: int64

In [23]:
one_sided_txns[one_sided_txns["status"]=="Fulfilled"]['asset_movement'].value_counts()

Polygon -> BSC       2045
BSC -> Polygon        892
Polygon -> Fantom     690
Polygon -> xDai       247
xDai -> Polygon       199
Fantom -> Polygon     170
BSC -> Fantom         167
Fantom -> BSC         110
xDai -> Fantom         57
xDai -> BSC            54
BSC -> xDai            47
Fantom -> xDai         28
Name: asset_movement, dtype: int64

In [24]:
one_sided_txns[one_sided_txns["status"]=="Fulfilled"]['asset_movement'].count()

4706

### Mapping Assets to Txns

In [196]:
chain_asset_data = {
   "Ethereum":{
      "0x0000000000000000000000000000000000000000":{
         "token":"ETH",
         "decimals":18
      },
      "0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0":{
         "token":"MATIC",
         "decimals":18
      },
      "0xb8c77482e45f1f44de1745f52c74426c631bdd52":{
         "token":"BNB",
         "decimals":18
      },
      "0x6b175474e89094c44da98b954eedeac495271d0f":{
         "token":"DAI",
         "decimals":18
      },
      "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48":{
         "token":"USDC",
         "decimals":6
      },
      "0xdac17f958d2ee523a2206206994597c13d831ec7":{
         "token":"USDT",
         "decimals":6
      }
   },
   "BSC":{
      "0x2170ed0880ac9a755fd29b2688956bd959f933f8":{
         "token":"ETH",
         "decimals":18
      },
      "0xcc42724c6683b7e57334c4e856f4c9965ed682bd":{
         "token":"MATIC",
         "decimals":18
      },
      "0x0000000000000000000000000000000000000000":{
         "token":"BNB",
         "decimals":18
      },
      "0x1af3f329e8be154074d8769d1ffa4ee058b1dbc3":{
         "token":"DAI",
         "decimals":18
      },
      "0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d":{
         "token":"USDC",
         "decimals":18
      },
      "0x55d398326f99059ff775485246999027b3197955":{
         "token":"USDT",
         "decimals":18
      }
   },
   "Polygon":{
      "0xfd8ee443ab7be5b1522a1c020c097cff1ddc1209":{
         "token":"ETH",
         "decimals":18
      },
      "0x0000000000000000000000000000000000000000":{
         "token":"MATIC",
         "decimals":18
      },
      "0xa649325aa7c5093d12d6f98eb4378deae68ce23f":{
         "token":"BNB",
         "decimals":18
      },
      "0x8f3cf7ad23cd3cadbd9735aff958023239c6a063":{
         "token":"DAI",
         "decimals":18
      },
      "0x2791bca1f2de4661ed88a30c99a7a9449aa84174":{
         "token":"USDC",
         "decimals":6
      },
      "0xc2132d05d31c914a87c6611c10748aeb04b58e8f":{
         "token":"USDT",
         "decimals":6
      }
   },
   "xDai":{
      "0xa5c7cb68cd81640d40c85b2e5ec9e4bb55be0214":{
         "token":"ETH",
         "decimals":18
      },
      "0x7122d7661c4564b7c6cd4878b06766489a6028a2":{
         "token":"MATIC",
         "decimals":18
      },
      "0xca8d20f3e0144a72c6b5d576e9bd3fd8557e2b04":{
         "token":"BNB",
         "decimals":18
      },
      "0x0000000000000000000000000000000000000000":{
         "token":"DAI",
         "decimals":18
      },
      "0xddafbb505ad214d7b80b1f830fccc89b60fb7a83":{
         "token":"USDC",
         "decimals":6
      },
      "0x4ecaba5870353805a9f068101a40e0f32ed605c6":{
         "token":"USDT",
         "decimals":6
      }
   },
   "Fantom":{
      "0x658b0c7613e890ee50b8c4bc6a3f41ef411208ad":{
         "token":"ETH",
         "decimals":18
      },
      "0x8d11ec38a3eb5e956b052f67da8bdc9bef8abf3e":{
         "token":"DAI",
         "decimals":18
      },
      "0x04068da6c83afcfa0e13ba15a6696662335d5b75":{
         "token":"USDC",
         "decimals":6
      },
      "0x049d68029688eabf473097a2fc38ef61633a3c7a":{
         "token":"USDT",
         "decimals":6
      }
   }
}

In [133]:
def find_asset_chain(query_asset_id):
    for chain, asset_data in chain_asset_data.items():
        for asset_id, token_data in asset_data.items():
            if asset_id == query_asset_id:
                print(chain, token_data)
find_asset_chain("0x8f3cf7ad23cd3cadbd9735aff958023239c6a063")

Polygon {'token': 'DAI', 'decimals': 18}


In [197]:
def asset_token_mapper(row, col_name):
    chain_asset_dict = chain_asset_data[row["chain"]]
    asset = chain_asset_dict[row[col_name]]
    return asset["token"]

In [198]:
def asset_decimal_mapper(row, col_name):
    chain_asset_dict = chain_asset_data[row["chain"]]
    asset = chain_asset_dict[row[col_name]]
    return asset["decimals"]

In [201]:
one_sided_txns["asset_token"] = one_sided_txns.apply(asset_token_mapper, args=(["sendingAssetId"]), axis=1)

In [202]:
one_sided_txns["decimals"] = one_sided_txns.apply(asset_decimal_mapper, args=(["sendingAssetId"]), axis=1)

In [34]:
def dollar_amount(row):
    dollar_value = int(row["amount"]) / 10**row["decimals"]
    return dollar_value

In [36]:
one_sided_txns["dollar_amount"] = one_sided_txns.apply(dollar_amount, axis=1)

In [37]:
one_sided_txns.dollar_amount.sum()

6492069.2619805

In [51]:
one_sided_txns

,amount,bidSignature,callDataHash,callTo,cancelCaller,cancelTransactionHash,chainId,expiry,fulfillCaller,fulfillTransactionHash,id,prepareCaller,prepareTransactionHash,preparedBlockNumber,preparedTimestamp,receivingAddress,receivingAssetId,receivingChainId,receivingChainTxManagerAddress,router,sendingAssetId,sendingChainFallback,sendingChainId,status,transactionId,user,chain,txn_type,asset_movement,asset_token,decimals,dollar_amount
0,1000000,0xd04b318889773a72ebe14686c71f4289a154e51c5404...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1630270871,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x5253f20e59f9c6d92b79832da14cbff382b7aada50b0...,0xe4ea45433c286c0c86402615bb86cb37136c3f3ae5d0...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0x2348eba581e9334f21774a903cf58b906891cf32cf4a...,18427665,1630011848,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,56,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,137,Fulfilled,0xe4ea45433c286c0c86402615bb86cb37136c3f3ae5d0...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,Polygon,single,Polygon -> BSC,USDC,6,1.000000
1,1000000,0xedf1c5c8c4e049da7c4ff5fba40bda127e23156d8d87...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1630274393,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x7d8dbb0eb60a5f4f95d6bb1061c9639a41fa5fc67717...,0x94441dc41596760a71ea9b6111baeec0177d6a148f54...,0xfcb5ae08aa85b908984ebaf2e8e584116f6ccba3,0xdfe8193755d3bac6f39a48d28ed621cf0e6a8823f8c0...,18429085,1630015230,0xfcb5ae08aa85b908984ebaf2e8e584116f6ccba3,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,56,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0xfcb5ae08aa85b908984ebaf2e8e584116f6ccba3,137,Fulfilled,0x94441dc41596760a71ea9b6111baeec0177d6a148f54...,0xfcb5ae08aa85b908984ebaf2e8e584116f6ccba3,Polygon,single,Polygon -> BSC,USDC,6,1.000000
7,1000000,0xb1b52f7621ddfa6ea72471b06b2fa87ea7d5ed0f3c55...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1630321028,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x00b8c745bfd86f182c4d50c3a3f344051cfb1e9542eb...,0x2643236cf25b357eb7b68eff00eed8b6603e5c6e35eb...,0x57e2a4ec7250d768220e8fa1110b210de917d0e9,0x5f904be7813cd650a8c6203872790e66a70747ef9507...,18448579,1630062028,0x57e2a4ec7250d768220e8fa1110b210de917d0e9,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,56,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x57e2a4ec7250d768220e8fa1110b210de917d0e9,137,Fulfilled,0x2643236cf25b357eb7b68eff00eed8b6603e5c6e35eb...,0x57e2a4ec7250d768220e8fa1110b210de917d0e9,Polygon,single,Polygon -> BSC,USDC,6,1.000000
8,100000000,0xe82b6064eb490ff866c551df34035e5a5072588ebbb4...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1630324402,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xf4d9d35b067b7efe8de8856b32522460830fded28112...,0x4632d845621cf800946da2c9f98ae7ab66bfe9bab0bc...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0xb0a3ec9502eabaf4e76f6cdcb255edf607b3bd037ca6...,18449678,1630065240,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,56,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,137,Fulfilled,0x4632d845621cf800946da2c9f98ae7ab66bfe9bab0bc...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,Polygon,single,Polygon -> BSC,USDC,6,100.000000
9,10000,0xb1d1853cc525a798c9f5a4d9469fc922879ef1592390...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,16

In [38]:
one_sided_txns[one_sided_txns['status']=="Fulfilled"]["dollar_amount"].sum()

5768368.565557634

In [73]:
new_df = one_sided_txns[["user","dollar_amount"]].groupby(['user'], sort=True, ).sum().reset_index()
new_df = new_df.sort_values(by = ['dollar_amount'], ascending=[False])
print(new_df.head(100))

                                            user  dollar_amount
2957  0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223  876004.000000
2605  0xdba3c7141c797dad1fdbafb60e0ca1b9230fc315  158000.000000
604   0x34b4df3f43d25073474e660712aea2482deb641b   95410.461243
2565  0xd8faec528b9fbf4fe3226d6beb8b14a4f9bad655   88064.000000
1478  0x7ee567d2ac5dca4c422281deb1e2334a805ba6c1   81500.000000
2129  0xb4146eceb339e8f573a014e03a24a40eace26c4b   80877.000000
2078  0xb07ac339291fdba30d2c75355aae22ed6f3d1c7c   77600.000000
3029  0xfe9d31b4161108ae6f8ef77e814148a38b9be370   76215.588000
1218  0x68b7d230072090924bb254650cf9df3dc1089745   74072.454000
1001  0x55f836ffd5b27329b07a47332ef00bfda7f4d62b   66077.077000
1191  0x661f257d22ba65fce0c01f5c255a21320504098a   62881.000000
2917  0xf4f6ecc5d5e34389ae180bb554043cd4cbea4253   60200.000000
90    0x07759bfb542723b5a89c5a927720bb8c299cd206   60185.000000
838   0x494115ac7a9332251249fc9637cc67761d43883a   60000.000000
400   0x22bf900bdfd4cdf0af25e142bf0d12f3

In [46]:
one_sided_txns[one_sided_txns['status']=="Fulfilled"].shape

(4706, 32)

In [48]:
one_sided_txns[one_sided_txns['status']=="Cancelled"].shape

(398, 32)

In [46]:
one_sided_txns[one_sided_txns['status']=="Fulfilled"].shape

(4706, 32)

In [104]:
x = one_sided_txns[["asset_movement","dollar_amount"]][one_sided_txns['status']=="Fulfilled"].groupby("asset_movement", axis=0).sum()
x.head()

,dollar_amount
asset_movement,
BSC -> Fantom,3.020563e+05
BSC -> Polygon,1.259154e+06
BSC -> xDai,2.456130e+04
Fantom -> BSC,1.278594e+05
Fantom -> Polygon,1.090145e+06


In [121]:
new_df = one_sided_txns[["asset_movement","dollar_amount"]].groupby(['asset_movement'], sort=True).sum().reset_index()
new_df = new_df.sort_values(by = ['dollar_amount'], ascending=[False])
new_df["dollar_amount"] = new_df["dollar_amount"].astype('int64')
new_df.rename({"dollar_amount":"Dollar Value($)"},axis=1, inplace=True)
print(new_df)

       asset_movement  Dollar Value($)
6      Polygon -> BSC          1497429
1      BSC -> Polygon          1449900
7   Polygon -> Fantom          1405606
4   Fantom -> Polygon          1279254
0       BSC -> Fantom           373392
3       Fantom -> BSC           132826
8     Polygon -> xDai           122711
11    xDai -> Polygon           119800
5      Fantom -> xDai            33009
9         xDai -> BSC            27851
10     xDai -> Fantom            25724
2         BSC -> xDai            24561


In [119]:
new_df.rename({"dollar_amount":"Dollar Value($)"}, axis=1,inplace=True)

In [40]:
for row in x.iterrows():
    print(row[0], end=" = ")
    print(row[1][1])

100 = 166771.80522999287
137 = 2468026.8525046837
250 = 1564221.643998964
56 = 1569348.2638239895


In [43]:
for row in x.iterrows():
    print(row[0], end=" = ")
    print(row[1][1])

BSC = 1585771.223553348
Fantom = 1246014.0917421093
Polygon = 2764298.7661431697
xDai = 172284.48411900012


In [218]:
for row in x.iterrows():
    print(row[0], end=" = ")
    print(row[1][1])

BSC = 1585771.223553348
Fantom = 1246014.0917421093
Polygon = 2763072.4851431698
xDai = 172284.48411900012


In [187]:
one_sided_txns[one_sided_txns.dollar_amount==100685.0]["transactionId"].values

array(['0xf24169c8557f87f3b0ad7ede8218a9a0551a72c43751fbd75130e5fe1bc247ed'],
      dtype=object)

In [183]:
one_sided_txns[one_sided_txns.dollar_amount==100685.0]["asset_movement"].values

array(['Fantom -> Polygon'], dtype=object)

In [186]:
one_sided_txns[one_sided_txns.dollar_amount==100685.0]["fulfillTransactionHash"].values

array(['0xcf664975c10de3a0e3781493d5b8c2d7ca32ee5d4880269458ad8be92705355b'],
      dtype=object)

In [185]:
one_sided_txns[one_sided_txns.dollar_amount==100685.0]

,amount,bidSignature,callDataHash,callTo,cancelCaller,cancelTransactionHash,chainId,expiry,fulfillCaller,fulfillTransactionHash,id,prepareCaller,prepareTransactionHash,preparedBlockNumber,preparedTimestamp,receivingAddress,receivingAssetId,receivingChainId,receivingChainTxManagerAddress,router,sendingAssetId,sendingChainFallback,sendingChainId,status,transactionId,user,chain,txn_type,asset_movement,asset_token,decimals,dollar_amount
156,100685000000,0x566d01c81fe7257d081568fada815d5d6e708cebc8ed...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,250,1630877968,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xcf664975c10de3a0e3781493d5b8c2d7ca32ee5d4880...,0xf24169c8557f87f3b0ad7ede8218a9a0551a72c43751...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0xf2533ae556052fa4357bfc0e314412f411e44aa4f40b...,16187471,1630618824,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,137,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x049d68029688eabf473097a2fc38ef61633a3c7a,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,250,Fulfilled,0xf24169c8557f87f3b0ad7ede8218a9a0551a72c43751...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,Fantom,single,Fantom -> Polygon,USDT,6,100685.0


### Date related analysis

In [210]:
from datetime import datetime,timedelta

In [82]:
pd.to_datetime("1630011089",unit="s")

Timestamp('2021-08-26 20:51:29')

In [74]:
one_sided_txns.preparedTimestamp.loc[0]

0    1630011848
0    1630011089
Name: preparedTimestamp, dtype: object

In [87]:
one_sided_txns["formatted_date"] = one_sided_txns["preparedTimestamp"].apply(lambda x: pd.to_datetime(x,unit="s"))

In [238]:
two_sided_txns["formatted_date"] = two_sided_txns["preparedTimestamp"].apply(lambda x: pd.to_datetime(x,unit="s"))

In [65]:
one_sided_txns.groupby("user").sum()

,decimals,dollar_amount
user,,
0x00000000000cd56832ce5dfbcbff02e7ec639bc9,18,7.974256
0x000000e28faa823d5b53ff6c2922c28335840375,6,149.000000
0x000bc1d6e8e17c21e312fa1ff054608d99334d53,6,3.597000
0x001ef02a3129690ef7b4ceb457efd6fba1658453,6,4.000000
0x00630a805ee7c191fff5e9b79dc82a42a9b3c444,36,2607.209700
...,...,...
0xffabeb614f7f98d8f4e1735e8031d740955d9dc9,18,26.214000
0xffb69e6abf30a6ab899a8371626aa9d06be8da8c,18,30.840000
0xffda8e1961d9e55ece7dde23deb06ef7a5b53e4c,6,152.000000


In [212]:
one_sided_txns[one_sided_txns["formatted_date"] > datetime.now()-timedelta(days=2)]

,amount,bidSignature,callDataHash,callTo,cancelCaller,cancelTransactionHash,chainId,expiry,fulfillCaller,fulfillTransactionHash,id,prepareCaller,prepareTransactionHash,preparedBlockNumber,preparedTimestamp,receivingAddress,receivingAssetId,receivingChainId,receivingChainTxManagerAddress,router,sendingAssetId,sendingChainFallback,sendingChainId,status,transactionId,user,chain,txn_type,asset_movement,asset_token,decimals,dollar_amount,formatted_date
3173,100000000,0xde79f3ed99c8a9ebeca8149b5e443d6001eadd65e9ee...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1631596051,0xe6a3dc2971532f5feb6b650e5dda2fe923d13af2,0x27ba7ef55d93a90a61581b051684d74a75938bbf4439...,0xf47b94a54f31c9a2d8973754708be42e818ca2e51469...,0x40e66c3ec2635fddbfd2c8097904ad5097b171d4,0xcf6569247e391bc091e6e8196ac152a86f606e928ee4...,19012529,1631419438,0x40e66c3ec2635fddbfd2c8097904ad5097b171d4,0x8d11ec38a3eb5e956b052f67da8bdc9bef8abf3e,250,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe6a3dc2971532f5feb6b650e5dda2fe923d13af2,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0x40e66c3ec2635fddbfd2c8097904ad5097b171d4,137,Fulfilled,0xf47b94a54f31c9a2d8973754708be42e818ca2e51469...,0x40e66c3ec2635fddbfd2c8097904ad5097b171d4,Polygon,single,Polygon -> Fantom,DAI,18,1.000000e-10,2021-09-12 04:03:58
3174,100000000,0xe80758796fcb9d0a19390555e4beb01e0524ae030dee...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1631599047,None,None,0xc0dd6eb00aa1bdc366d8fba21451496d48e4f7497a6a...,0x40e66c3ec2635fddbfd2c8097904ad5097b171d4,0x6fde706d51f4aa89ba156279b2e419ff310508d7d816...,19013601,1631422418,0x40e66c3ec2635fddbfd2c8097904ad5097b171d4,0x8d11ec38a3eb5e956b052f67da8bdc9bef8abf3e,250,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe6a3dc2971532f5feb6b650e5dda2fe923d13af2,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0x40e66c3ec2635fddbfd2c8097904ad5097b171d4,137,Prepared,0xc0dd6eb00aa1bdc366d8fba21451496d48e4f7497a6a...,0x40e66c3ec2635fddbfd2c8097904ad5097b171d4,Polygon,single,Polygon -> Fantom,DAI,18,1.000000e-10,2021-09-12 04:53:38
3175,1226281000,0x107d12f51b8063b98df9b45a85dd432134293d1a139c...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1631701892,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x92fb5f5e7d03454bb1ebcdc354b5406628d6b9ed1e0e...,0x09022f23fd1b6eb09217c1bf27181c66e5cee6a7fbe9...,0x1b9ac7594436fd0f8c19c2193e5dc0d77b793433,0x79aa12fc4fb7f9f6ca0484435e73fec09d0ed1d6b91a...,19022867,1631444913,0x1b9ac7594436fd0f8c19c2193e5dc0d77b793433,0x55d398326f99059ff775485246999027b3197955,56,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,0x1b9ac7594436fd0f8c19c2193e5dc0d77b793433,137,Fulfilled,0x09022f23fd1b6eb09217c1bf27181c66e5cee6a7fbe9...,0x1b9ac7594436fd0f8c19c2193e5dc0d77b793433,Polygon,single,Polygon -> BSC,USDT,6,1.226281e+03,2021-09-12 11:08:33


In [214]:
two_sided_txns.head()

,amount,bidSignature,callDataHash,callTo,cancelCaller,cancelTransactionHash,chainId,expiry,fulfillCaller,fulfillTransactionHash,id,prepareCaller,prepareTransactionHash,preparedBlockNumber,preparedTimestamp,receivingAddress,receivingAssetId,receivingChainId,receivingChainTxManagerAddress,router,sendingAssetId,sendingChainFallback,sendingChainId,status,transactionId,user,chain,txn_type
0,1000000,0xd04b318889773a72ebe14686c71f4289a154e51c5404...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1630270871,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x5253f20e59f9c6d92b79832da14cbff382b7aada50b0...,0xe4ea45433c286c0c86402615bb86cb37136c3f3ae5d0...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0x2348eba581e9334f21774a903cf58b906891cf32cf4a...,18427665,1630011848,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,56,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,137,Fulfilled,0xe4ea45433c286c0c86402615bb86cb37136c3f3ae5d0...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,Polygon,single
1,1000000,0xedf1c5c8c4e049da7c4ff5fba40bda127e23156d8d87...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1630274393,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x7d8dbb0eb60a5f4f95d6bb1061c9639a41fa5fc67717...,0x94441dc41596760a71ea9b6111baeec0177d6a148f54...,0xfcb5ae08aa85b908984ebaf2e8e584116f6ccba3,0xdfe8193755d3bac6f39a48d28ed621cf0e6a8823f8c0...,18429085,1630015230,0xfcb5ae08aa85b908984ebaf2e8e584116f6ccba3,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,56,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0xfcb5ae08aa85b908984ebaf2e8e584116f6ccba3,137,Fulfilled,0x94441dc41596760a71ea9b6111baeec0177d6a148f54...,0xfcb5ae08aa85b908984ebaf2e8e584116f6ccba3,Polygon,single
2,19990000,0xce9a010d26ac0b8d9ef4637d98df1fc484a5fc7a8d7b...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xd48dd4fbea512dd2541e263fbbafc4146129b5879251...,137,1630206428,None,None,0x6470c3f7f74ac89eeb429f29f88b3879d87485414500...,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x6bd36986524275d206762681b4df34dac88b5f59afbb...,18446015,1630055897,0xe98e4f51ca28b4851101aa83e22e59abdd54e7fc,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,137,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,0xe98e4f51ca28b4851101aa83e22e59abdd54e7fc,56,Cancelled,0x6470c3f7f74ac89eeb429f29f88b3879d87485414500...,0xe98e4f51ca28b4851101aa83e22e59abdd54e7fc,Polygon,repeat
3,999,0xe9c539ed9b41ec83074918a977f41f6040dcff8074fd...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1630186495,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x2eeef7fe2b59cc8c1da638d45711e9686128b4cf7f92...,0x837f8d0e378ea9da3e137fe17a80205ad7bfee538c73...,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xf73549b97eb534a5ce02ee82c27adde412ac6c2faf69...,18446015,1630055897,0x65f6f29d3eb871254d71a79cc4f74db3aaf3b86e,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,137,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d,0x65f6f29d3eb871254d71a79cc4f74db3aaf3b86e,56,Fulfilled,0x837f8d0e378ea9da3e137fe17a80205ad7bfee538c73...,0x65f6f29d3eb871254d71a79cc4f74db3aaf3b86e,Polygon,repeat
4,99950,0xca9070e5842e883dc5f69159e2672374fb9aa4b9d3d4...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1630216692,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x86ba9a9e5f508454c8520574912a23b25cde6255ef9f...,0xc3cfc2fc147d488420c3da4dc4c818ce72792e247e2d...,0xe1dd7b28ab085978ead65aff92b

In [233]:
bidid="0xe83bdb80b3ca90ed597fe4fad078de2cd240607d53747b51760e2719c12a093f63d1a75d9d0dbe0a2ba6ac245ccf2321a8082579e271112fdbeaad8ca7af27711b"

In [239]:
two_sided_txns[two_sided_txns.receivingAddress=="0x4752a32843b661FEDC525C979Aaa01Fa54d31366".lower()]

,amount,bidSignature,callDataHash,callTo,cancelCaller,cancelTransactionHash,chainId,expiry,fulfillCaller,fulfillTransactionHash,id,prepareCaller,prepareTransactionHash,preparedBlockNumber,preparedTimestamp,receivingAddress,receivingAssetId,receivingChainId,receivingChainTxManagerAddress,router,sendingAssetId,sendingChainFallback,sendingChainId,status,transactionId,user,chain,txn_type,asset_movement,formatted_date
3029,200000000000000000,0x7c2a8087577e476fed214ec6c76179f6ff803847d436...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1631189000,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xb1c8c92a707d7e3229487282bb90a676104533bb4e83...,0x144c7d9570851fc0c053150125079edd156a6b4ea53f...,0x4752a32843b661fedc525c979aaa01fa54d31366,0x64f1e20b58adffee60a52cc113aec275427f5ced7d30...,18809304,1630931002,0x4752a32843b661fedc525c979aaa01fa54d31366,0x0000000000000000000000000000000000000000,100,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0x4752a32843b661fedc525c979aaa01fa54d31366,137,Fulfilled,0x144c7d9570851fc0c053150125079edd156a6b4ea53f...,0x4752a32843b661fedc525c979aaa01fa54d31366,Polygon,single,Polygon -> xDai,2021-09-06 12:23:22
3088,200000000000000000,0x3986b12bda7f56909a130f5c677fc57f4e9a08c813c7...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,137,1631193933,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8ca1a4e9d48b3b6d1b49dc411d303741b13512cc62ac...,0x099f2f7b4800eaa392dd017cee8145968b983e6ad712...,0x4752a32843b661fedc525c979aaa01fa54d31366,0xb7825ea3b34f8923eb9252dbcd16952e2580797fe671...,18811078,1630934800,0x4752a32843b661fedc525c979aaa01fa54d31366,0x0000000000000000000000000000000000000000,100,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0x4752a32843b661fedc525c979aaa01fa54d31366,137,Fulfilled,0x099f2f7b4800eaa392dd017cee8145968b983e6ad712...,0x4752a32843b661fedc525c979aaa01fa54d31366,Polygon,single,Polygon -> xDai,2021-09-06 13:26:40
490,199900000000000000,0x7c2a8087577e476fed214ec6c76179f6ff803847d436...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,100,1631102600,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xdf8fdad25a07d39986b3ddb4c4177619b7d07f2db31a...,0x144c7d9570851fc0c053150125079edd156a6b4ea53f...,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8f1b47e62148b5dda46d4f2ae6c99d3e6ff43992d48f...,17954914,1630931040,0x4752a32843b661fedc525c979aaa01fa54d31366,0x0000000000000000000000000000000000000000,100,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0x4752a32843b661fedc525c979aaa01fa54d31366,137,Fulfilled,0x144c7d9570851fc0c053150125079edd156a6b4ea53f...,0x4752a32843b661fedc525c979aaa01fa54d31366,xDai,repeat,Polygon -> xDai,2021-09-06 12:24:00
503,199900000000000000,0x3986b12bda7f56909a130f5c677fc57f4e9a08c813c7...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,100,1631107533,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xd91911412028dea8ed9725ceedc6bf6df53f00047088...,0x099f2f7b4800eaa392dd017cee8145968b983e6ad712...,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x73ec62e35fe1d901d3fcc16acf22cb25cf54d0e9253d...,17955602,1630934865,0x4752a32843b661fedc525c979aaa01fa54d31366,0x0000000000000000000000000000000000000000,100,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0x4752a32843b661fedc525c979aaa01fa54d31366,137,Fulfilled,0x099f2f7b4800eaa392dd017cee8145968b983e6ad712...,0x4752a32843b661fedc525c979aaa01fa54d31366,xDai,repeat,Polygon -> xDai,2021-09-06 13:27:45


### Transaction 1 from Polygon to xDai 
<br>Prepared Timestamp at **12:23:22** on Polygon hash: 0x64f1e20b58adffee60a52cc113aec275427f5ced7d30829e18488fa0a15effdf
<br>Prepared Timestamp at **12:24:00** on xDai hash: 0x8f1b47e62148b5dda46d4f2ae6c99d3e6ff43992d48fea04d5ea60cc5ac52ae7
<br>Fulfilled Timestamp at **12:24:15** on xDai hash: 0xdf8fdad25a07d39986b3ddb4c4177619b7d07f2db31a2f3ea0c61cd45f185c14
<br>Fulfilled Timestamp at **12:27:29** on Polygon 0xb1c8c92a707d7e3229487282bb90a676104533bb4e83744ad9c5f0dc0ff5725d

### Transaction 2 from Polygon to xDai 
<br>Prepared Timestamp at **13:26:40** on Polygon hash: 0xb7825ea3b34f8923eb9252dbcd16952e2580797fe671ee2722ad0042c4167bb0
<br>Prepared Timestamp at **13:27:45** on xDai hash: 0x73ec62e35fe1d901d3fcc16acf22cb25cf54d0e9253db98b96a0e0db20e3aa7c
<br>Fulfilled Timestamp at **13:28:10** on xDai hash: 0xd91911412028dea8ed9725ceedc6bf6df53f0004708831bccc139cb115c27ac1
<br>Fulfilled Timestamp at **13:31:30** on Polygon 0x8ca1a4e9d48b3b6d1b49dc411d303741b13512cc62ac5cc65e22ee3c690c110c

In [245]:
two_sided_txns[two_sided_txns.receivingAddress=="0x4752a32843b661FEDC525C979Aaa01Fa54d31366".lower()]["prepareTransactionHash"].values

array(['0x64f1e20b58adffee60a52cc113aec275427f5ced7d30829e18488fa0a15effdf',
       '0xb7825ea3b34f8923eb9252dbcd16952e2580797fe671ee2722ad0042c4167bb0',
       '0x8f1b47e62148b5dda46d4f2ae6c99d3e6ff43992d48fea04d5ea60cc5ac52ae7',
       '0x73ec62e35fe1d901d3fcc16acf22cb25cf54d0e9253db98b96a0e0db20e3aa7c'],
      dtype=object)

In [246]:
two_sided_txns[two_sided_txns.receivingAddress=="0x4752a32843b661FEDC525C979Aaa01Fa54d31366".lower()]["fulfillTransactionHash"].values

array(['0xb1c8c92a707d7e3229487282bb90a676104533bb4e83744ad9c5f0dc0ff5725d',
       '0x8ca1a4e9d48b3b6d1b49dc411d303741b13512cc62ac5cc65e22ee3c690c110c',
       '0xdf8fdad25a07d39986b3ddb4c4177619b7d07f2db31a2f3ea0c61cd45f185c14',
       '0xd91911412028dea8ed9725ceedc6bf6df53f0004708831bccc139cb115c27ac1'],
      dtype=object)

In [227]:
two_sided_txns.bidSignature.value_counts()

0x103d34a3706e258bb0d1b8985b84344bf70e45fddf3ec7802f802994e34ba24328e1211516838d658b7ef977eb2947a87d0bf3e25a098e925b85e96c424c659b1b    2
0xe83bdb80b3ca90ed597fe4fad078de2cd240607d53747b51760e2719c12a093f63d1a75d9d0dbe0a2ba6ac245ccf2321a8082579e271112fdbeaad8ca7af27711b    2
0xf5d1e371223b262af6f46de3be4ea93710da2c8ee03f76807464cad7cc190fac44ea2bda53eff2640dfe8112377d7214841f47fd8c57cb9da2a3700aee5808d71c    2
0x417a10c32e0cbef2296b8da8d884a72456d8950a8c3d678d5c4fdaacfa5aed361fd90af21ad6932f1aa5aff6bcafdfeb292ea8bc92a9123012c3b2c12530535f1b    2
0xe43eaae26e734878af86b1541630fb8d0046b93eb601eb6b022f2cd95367f6a04b8eded2e1b1678d04544d66159e614f0d3f7583f96c6b2a109ea9e5392910671b    2
                                                                                                                                       ..
0x0a8cea64781f24132f2469bb259275eaad125a8c18faec26a7f54efd116f41df588d264523b9435833448519bb02da1097985f916d8ce88b0c7a83c3abe74f9b1b    1
0xa7f68abfa9ff7aa6b33a3e948e7eb6ad

In [218]:
bid_dict = dict(two_sided_txns.bidSignature.value_counts())

In [224]:
two_sided_txns.shape

(10260, 28)

In [223]:
count = 0
n = 0
for sig in bid_dict.items():
    if sig[1] == 2:
        count+=1
    else:
        n+=1
#     break
print(count,n)

5089 82


## Router Data

### Combining Data

In [193]:
matic_routers["chain"] = "Polygon"

bsc_routers["chain"] = "BSC"

xdai_routers["chain"] = "xDai"

fantom_routers["chain"] = "Fantom"

In [194]:
all_routers = pd.concat([matic_routers, bsc_routers, xdai_routers, fantom_routers])

In [207]:
all_routers["asset_token"] = all_routers.apply(asset_token_mapper, args=(["asset_id"]), axis=1)
all_routers["decimals"] = all_routers.apply(asset_decimal_mapper, args=(["asset_id"]), axis=1)
all_routers["dollar_amount"] = all_routers.apply(dollar_amount, axis=1)

In [208]:
all_routers

,router_id,asset_id,amount,chain,asset_token,decimals,dollar_amount
0,0x29a519e21d6a97cdb82270b69c98bac6426cdcf9,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,97696466480,Polygon,USDC,6,97696.466480
1,0x29a519e21d6a97cdb82270b69c98bac6426cdcf9,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,67532401966040974476932,Polygon,DAI,18,67532.401966
2,0x29a519e21d6a97cdb82270b69c98bac6426cdcf9,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,118429226694,Polygon,USDT,6,118429.226694
3,0x709a10a487aa2fc85845b9c8a56ad28d5fa44914,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,1087310000000000000,Polygon,DAI,18,1.087310
4,0xaca3ba0cf90431fccb0f03e045d362350a9455b1,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,100000000000000000,Polygon,DAI,18,0.100000
5,0xdf011911095ac02469576b85ec4e33b48c2f0766,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0,Polygon,USDC,6,0.000000
6,0xdf011911095ac02469576b85ec4e33b48c2f0766,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,0,Polygon,USDT,6,0.000000
7,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0,Polygon,USDC,6,0.000000
8,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063,0,Polygon,DAI,18,0.000000
9,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,0,Polygon,USDT,6,0.000000


In [247]:
all_routers.dollar_amount.sum()

371507.9785985384